In [1]:
import sys
import pandas as pd
import numpy as np
import scipy
import tensorflow as tf
import sklearn 
import pymongo
import psycopg2
import warnings 
warnings.filterwarnings(action='ignore')

print('python ==> ',sys.version)
print('pandas ==>',pd.__version__)
print('numpy ==>',np.__version__)
print('scipy ==>',scipy.__version__)
print('tensorflow ==> ',tf.__version__)
print('scikit-learn ==> ',sklearn.__version__)
print('Pymongo -->',pymongo.__version__)
print('Psycopg2 -->',psycopg2.__version__)

python ==>  3.9.21 (main, Dec 11 2024, 16:35:24) [MSC v.1929 64 bit (AMD64)]
pandas ==> 2.2.3
numpy ==> 1.23.1
scipy ==> 1.10.1
tensorflow ==>  2.7.0
scikit-learn ==>  1.5.2
Pymongo --> 4.10.1
Psycopg2 --> 2.9.10 (dt dec pq3 ext lo64)


In [2]:
import datetime
start_date = '2023-12-01 00:00:00'
end_date = datetime.datetime.now().strftime('%Y-%m-%d 23:59:59')

print(start_date)
print(end_date)

2023-12-01 00:00:00
2025-05-15 23:59:59


#### 1.) PostgreSQL 데이터수집
- 장비(maker) 1,2,3 Raw Data 수집
- 생산(Production) & 생산결과(Result) 개별 수집
- DB정보 Private 처리 (코드 저장용)

In [ ]:
import datetime
pgdb = psycopg2.connect( # --> 기존 연결정보 Private 처리 
                 host='Private Host',
                dbname='Private DB',
                user='Company DB',
                password='password',
                port=15432)
print(pgdb)
cursor = pgdb.cursor()
query = f"""
SELECT * 
FROM temp.view_mass_if_private_maker 
WHERE shotdatetime >= '{start_date}' 
  AND shotdatetime <= '{end_date}'
"""
cursor.execute(query)
column_names = [desc[0] for desc in cursor.description]
result = cursor.fetchall() # # .fetchone() , fetchall(), fetchmany()
df_production = pd.DataFrame(result)
if len(df_production) >0:
    df_production.columns = column_names
    df_production = df_production.sort_values(by='shotdatetime').reset_index(drop=True)
    df_production.to_csv('Maker_Production_Postgre.csv',encoding='cp949',index=False)
    cursor.close()
else:
    cursor.close()

In [ ]:
pgdb = psycopg2.connect( # --> 기존 연결정보 Private 처리 
                 host='Private Host',
                dbname='Private DB',
                user='Company DB',
                password='password',
                port=15432)
cursor = pgdb.cursor()
print(pgdb)
query = f"""
SELECT * 
FROM temp.view_p_results_private_maker
WHERE m_shotdatetime >= '{start_date}' 
  AND m_shotdatetime <= '{end_date}'
"""
cursor.execute(query) 
column_names = [desc[0] for desc in cursor.description]
result=cursor.fetchall() # .fetchone() , fetchall(), fetchmany()
column_names = [desc[0] for desc in cursor.description]
df_result = pd.DataFrame(result)
if len(df_result) >0:
    df_result.columns = column_names
    df_result = df_result.sort_values(by='m_shotdatetime').reset_index(drop=True)
    df_result = df_result[df_result['m_shotdatetime']!='1900-01-01 00:00:00'].reset_index(drop=True)
    df_result.to_csv('Maker_Result_Postgre.csv',encoding='cp949',index=False)
    cursor.close()
else:
    cursor.close()